## FOMC Meeting End Dates

Preamble: Scientific findings are by definition reproducible by controlled experiment. This notebook validates results in this paper: http://faculty.haas.berkeley.edu/vissing/cycle_paper_cieslak_morse_vissingjorgensen.pdf

i. Web Scrape FOMC Meeting End Dates

In [3]:
import requests
from lxml import html
from datetime import *
import pandas as pd
import numpy as np

Recent Meetings: https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm

In [4]:
url = 'https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm'
fomccalendars = html.fromstring(requests.get(url).content)

In [25]:
fomc_dates = []
for panel in fomccalendars.xpath(".//div[@class='panel panel-default']"):
    year = int(panel.xpath(".//div[@class='panel-heading']//text()")[0][0:4])
    months = panel.xpath(".//div[contains(@class, 'month')]//text()")
    dates = panel.xpath(".//div[contains(@class, 'date')]/text()")
    m = [ datetime.strptime(month.split('/')[-1][0:3], '%b').month for month in months ]
    d = [ int(day.split('-')[-1].rstrip('*')) for day in dates if not "unscheduled" in day ]
    fomc_dates += [ datetime(year, m, d).date() for m,d in zip(m, d) ]    

Historical Meetings: https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm

In [28]:
url = 'https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm'
fomc_historical_year = html.fromstring(requests.get(url).content)
fomchistorical = [ 'https://www.federalreserve.gov'+href for href in fomc_historical_year.xpath(".//div[@id='article']//a/@href") ]

In [30]:
dates = []
for url in fomchistorical:
    historical = html.fromstring(requests.get(url).content)
    dates += historical.xpath(".//h5/text()")

In [31]:
meetings = [ meeting.split() for meeting in dates if "Meeting" in meeting ]
years = [ int(meeting[-1]) for meeting in meetings]
months = [ datetime.strptime(date[0], '%B').month if len(date)==5 else datetime.strptime(date[0], '%B').month+1 for date in meetings ]
days = [ int(date[1].split('-')[-1]) if len(date)==5 else 1 for date in meetings ]
fomc_dates += [ datetime(y, m, d).date() for y,m,d in zip(years, months, days) ]

In [36]:
# 2003 Sep
fomc_dates.sort()
fomc_df = pd.DataFrame(fomc_dates, columns=['fomc_date'], dtype='datetime64[ns]')

ii. Calculate FOMC Calendar

In [68]:
calendar = pd.bdate_range(start=min(fomc_df['fomc_date']), end=max(fomc_df['fomc_date']))
last_fomc = [ max(fomc_df[fomc_df['fomc_date'] <= date]['fomc_date']) for date in calendar ]
next_fomc = [ min(fomc_df[fomc_df['fomc_date'] >= date]['fomc_date']) for date in calendar ]
days_since = [ np.busday_count(last_fomc[d], calendar[d]) for d in range(len(calendar)) ]
days_before = [ -np.busday_count(calendar[d], next_fomc[d]) for d in range(len(calendar)) ]
fomc_day = [ days_before[d] if days_before[d] >= -6 else days_since[d] for d in range(len(calendar)) ]
fomc_week = [ (fomc_day[d] + 1) // 5 for d in range(len(calendar))]
fomc_calendar = pd.DataFrame(data={'fomc_day': fomc_day, 'fomc_week': fomc_week}, index=calendar)
fomc_calendar.index.name = 'date'

iii. Returns

In [ ]:
# 5day logic data in QC FOMC

In [73]:
five_day = pd.read_csv('5day.csv', index_col=0, parse_dates=[0], infer_datetime_format=True)
df = fomc_calendar.join(five_day).dropna()

In [76]:
fomc = df[df['fomc_day'] < 34]
group = fomc.groupby('fomc_day')

In [77]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [78]:
source = ColumnDataSource(group)

hover = HoverTool(tooltips=[("(x,y)", "(@fomc_day, $y{0.00%})")], mode='vline')
p = figure(title='Stock returns of the FOMC cycle, 1998-2018', plot_width=800, plot_height=400) 
p.add_tools(hover)

p.line(x='fomc_day', y='5-day_mean', source=source)
p.circle(x='fomc_day', y='5-day_mean', source=source)
p.xaxis.axis_label = 'Days since FOMC meeting (weekends excluded)'
p.yaxis.axis_label = 'Avg. 5-day stock return, t to t+4'
labels = LabelSet(x='fomc_day', y='5-day_mean', text='fomc_day', source=source,
                  x_offset=5, y_offset=-10, text_font_size='10pt')
p.add_layout(labels)

show(p)